**Preprocessing Snort Logs**

In [3]:
#Importing required Libraries
from datetime import datetime

In [4]:
#Importing Files(logs) 
file_handler = open("ddos.txt", "r")
output_handler = open("tcplog.csv", "w")
# Columns
columns = ["datetime", "duration", "protocol", "src", "src_port", "dst", "dst_port", "src_bytes", "dst_bytes"]
str_columns = ", ".join(columns) + ",\n"
output_handler.write(str_columns)

82

In [5]:
file_lines = file_handler.readlines()

In [6]:
udp_line_num = -1
for idx, line in enumerate(file_lines):
    if "UDP connection info" in line:
        udp_line_num = idx
        break
if udp_line_num == -1:
    udp_line_num = len(file_lines)



In [7]:
udp_line_num

270068

In [8]:
file_lines[0]

"1 arg remaining, starting with 'snort.log.1650088386'\n"

In [9]:
def processTCP(tcp_connection_lines, output_handler):
    reset_bias = 0
    if "resets sent" in tcp_connection_lines[13]:
        reset_bias = 1
    sr_no = tcp_connection_lines[0].strip().split()[2]
    start_src = tcp_connection_lines[1].strip().split(":")[1].strip()
    start_src_port = tcp_connection_lines[1].strip().split(":")[-1]
    end_src = tcp_connection_lines[2].strip().split(":")[1].strip()
    end_src_port = tcp_connection_lines[2].strip().split(":")[-1]

    start_time = " ".join(tcp_connection_lines[4].strip().split()[3:])
    start_time = datetime.strptime(start_time, "%b %d %H:%M:%S.%f %Y")
    start_time = datetime.strftime(start_time, "%m/%d-%H:%M:%S.%f")[:-3]
    duration = tcp_connection_lines[6].strip().split()[2]
    protocol = "TCP"
    src_bytes = tcp_connection_lines[18+reset_bias].strip().split()[3]
    dst_bytes = tcp_connection_lines[18+reset_bias].strip().split()[7]

    output_handler.write(f"{start_time}, {duration}, {protocol}, {start_src}, {start_src_port}, {end_src}, {end_src_port}, {src_bytes}, {dst_bytes}.\n")

In [10]:
for i in range(0, -1):
    print(i)

In [11]:
for idx in range(0, udp_line_num):
    line = file_lines[idx]
    if ("arg remaining, starting with" in line or 
        "Ostermann's tcptrace" in line or  
        "packets seen," in line or 
        "elapsed wallclock time" in line or 
        "trace file elapsed time" in line or 
        "TCP connection info" in line or
        "TCP connection traced" in line):
        #print(line)
        continue
    
    elif "TCP connection " in line:

        idx2 = idx
        
        while True:
            try:
                line2 = file_lines[idx2]
            except:
                break
            if "============================" in line2:
                break
            idx2 += 1
        
        
        x = file_lines[idx: idx2]
        #print(x)
        processTCP(x, output_handler)
output_handler.close()